# OpenAI Agents

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.4/249.4 kB 6.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 11.8 MB/s eta 0:00:00


In [2]:
# to use non-openai models, e.g., from Hugging Face
!pip install -Uq "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 94.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 98.5 MB/s eta 0:00:00:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.67.1 which is incompatible.


In [3]:
# enable async in notebook
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
import getpass

# set default model for agents
os.environ["OPENAI_DEFAULT_MODEL"] = "gpt-5-mini"

# openai API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Quickstart

In [5]:
from agents import Agent, function_tool, Runner

@function_tool
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-5-mini",
    tools=[get_weather],
)

result = await Runner.run(agent, "What's the weather in New York?")

print(result.final_output)

Sun warms glass and stone,  
Blue sky folds the city bright—  
Sunny streets hum life.


In [6]:
print(type(result))

from pprint import pprint
pprint(result.__dict__, indent=1)

<class 'agents.result.RunResult'>
{'_last_agent': Agent(name='Haiku agent',
                      handoff_description=None,
                      tools=[FunctionTool(name='get_weather',
                                          description='returns weather info '
                                                      'for the specified city.',
                                          params_json_schema={'additionalProperties': False,
                                                              'properties': {'city': {'title': 'City',
                                                                                      'type': 'string'}},
                                                              'required': ['city'],
                                                              'title': 'get_weather_args',
                                                              'type': 'object'},
                                          on_invoke_tool=<function function_tool.<locals>._create

## Non Open-AI models

Use Hugging Face models with OpenAI Agents SDK.

Just add the HF_TOKEN env variable. And set the model param to:

```
litellm/huggingface/<provider>/<hf_org_or_user>/<hf_model>
```

In [7]:
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("Enter your Hugging Face token: ")

In [27]:
# using kimi-k2-thinking

from agents import Agent, Runner, ModelSettings
from agents.extensions.models.litellm_model import LitellmModel

model_minimax = LitellmModel(
    model="huggingface/novita/MiniMaxAI/MiniMax-M2.1",
    api_key=os.environ["HF_TOKEN"],
)

agent_kimi = Agent(
    name="Kimi agent",
    instructions="Always respond in haiku form",
    # model="litellm/huggingface/novita/MiniMaxAI/MiniMax-M2.1",
    tools=[get_weather],
    model=model_minimax,
    # optional, for tracing (requires openai API key)
    model_settings=ModelSettings(include_usage=True,),
)

result = await Runner.run(agent_kimi, "What's the weather in New York?")

print(result.final_output)

Sun shines in New York
Golden light bathes the streets
City walks in warmth


In [10]:
pprint(result.__dict__)

{'_last_agent': Agent(name='Kimi agent',
                      handoff_description=None,
                      tools=[FunctionTool(name='get_weather',
                                          description='returns weather info '
                                                      'for the specified city.',
                                          params_json_schema={'additionalProperties': False,
                                                              'properties': {'city': {'title': 'City',
                                                                                      'type': 'string'}},
                                                              'required': ['city'],
                                                              'title': 'get_weather_args',
                                                              'type': 'object'},
                                          on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_

## Agents

### Structured Output

- Use the `output_type` param with Pydantic objects (or dataclasses, lists, TypedDicts, etc.)
- When using non-openai models, remember to check that they support both Structured Ouput AND tool calling.

In [28]:
from pydantic import BaseModel
from agents import Agent


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

model = LitellmModel(
    model="huggingface/novita/zai-org/GLM-4.7",
    api_key=os.environ["HF_TOKEN"],
)

agent = Agent(
    name="Calendar extractor",
    instructions="Extract calendar events from text",
    output_type=CalendarEvent,
    model=model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(
    agent,
    "Extract the event from the following text: 'Meeting with Alice and Bob on July 5th.'",
)

print(result.final_output)

name='Meeting with Alice and Bob' date='July 5th' participants=['Alice', 'Bob']


### Multi-Agent systems

Two main architectures:
- Manager (agents as tools): A central manager/orchestrator invokes specialized sub‑agents as tools and retains control of the conversation.
- Handoffs: Peer agents hand off control to a specialized agent that takes over the conversation. This is decentralized.


#### Agents Handoffs

In [11]:
from agents import Agent

glm_model = LitellmModel(
    model="huggingface/novita/zai-org/GLM-4.7",
    api_key=os.environ["HF_TOKEN"],
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(
    triage_agent,
    "Can you explain the causes of World War II?",
)

print(result.final_output)

The causes of World War II are complex and rooted deeply in the aftermath of World War I. While the immediate trigger was the invasion of Poland, the conflict was the result of unresolved political issues, economic instability, and the rise of aggressive totalitarian regimes during the 1930s.

Here is a breakdown of the primary causes and context:

### 1. The Aftermath of World War I
The peace settlement that ended WWI, particularly the **Treaty of Versailles (1919)**, left a lasting resentment in Germany.
*   **War Guilt Clause:** Germany was forced to accept full responsibility for the war, which was a humiliation for the German people.
*   **Reparations:** Germany was hit with massive financial reparations it couldn't afford, leading to hyperinflation and economic collapse in the early 1920s.
*   **Territorial Losses:** Germany lost significant territory (such as Alsace-Lorraine to France and the Polish Corridor) and was stripped of its overseas colonies and military power.

### 2. 

#### Agents As Tools

In [49]:
manager_agent = Agent(
    name="Manager Agent",
    instructions="You manage a team of agents to answer user questions effectively.",
    tools=[
        history_tutor_agent.as_tool(
            tool_name="history_tutor_agent",
            tool_description="Handles historical queries",
        ),
        math_tutor_agent.as_tool(
            tool_name="math_tutor_agent",
            tool_description="Handles math questions",
        ),
    ],
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(
    manager_agent,
    "Can you explain the causes of World War II?",
)

print(result.final_output)

The causes of World War II were complex and rooted in unresolved issues from World War I, economic instability, and the rise of aggressive totalitarian regimes. Here's a breakdown:

## 1. The Treaty of Versailles (1919)
- Imposed harsh penalties on Germany: accepted full responsibility, paid massive reparations, lost territory, faced military restrictions
- Created deep resentment and humiliation, fueling radical political movements

## 2. Rise of Totalitarianism and Fascism
- **Adolf Hitler (Germany):** Promoted German superiority (*Lebensraum*) and sought to overturn the Treaty
- **Benito Mussolini (Italy):** Pursued imperial ambitions in Africa and the Mediterranean
- **Military Japan:** Sought domination of East Asia to secure resources

## 3. Failure of the League of Nations
- Lacked enforcement power and key member support
- Weak response to Japan's invasion of Manchuria (1931) and Italy's invasion of Ethiopia (1935)
- Emboldened aggressors by showing they could act without conse

## Tools

### Pre-built Tools

OpenAI offers a few built-in tools when using the OpenAIResponsesModel:

- The `WebSearchTool` lets an agent search the web.
- The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.
- The `ComputerTool` allows automating computer use tasks.
- The `CodeInterpreterTool` lets the LLM execute code in a sandboxed environment.
- The `HostedMCPTool` exposes a remote MCP server's tools to the model.
- The `ImageGenerationTool` generates images from a prompt.
- The `LocalShellTool` runs shell commands on your machine.

In [32]:
from agents import Agent, Runner, WebSearchTool

agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(),
    ],
)

async def main():
    result = await Runner.run(agent, "Who is the current president of the United States as of 2026?")
    print(result.final_output)

await main()


As of January 6, 2026 the President of the United States is Donald J. Trump (47th President), inaugurated January 20, 2025. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Second_presidency_of_Donald_Trump?utm_source=openai))


### Custom Tools

You can define custom tools that the agent can use.

In [36]:
import json
from typing_extensions import TypedDict, Any
from agents import Agent, FunctionTool, RunContextWrapper, function_tool


class Location(TypedDict):
    lat: float
    long: float

@function_tool  
async def fetch_weather(location: Location) -> str:
    
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return f"The weather in {location['lat']}, {location['long']} is sunny"


@function_tool(name_override="fetch_data")  
def read_file(ctx: RunContextWrapper[Any], path: str, directory: str | None = None) -> str:
    """Read the contents of a file.

    Args:
        path: The path to the file to read.
        directory: The directory to read the file from.
    """
    # In real life, we'd read the file from the file system
    return "Hello, World!"


agent = Agent(
    name="Assistant",
    tools=[fetch_weather, read_file],  
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(agent, "What is the weather in San Francisco?")
print(result.final_output)

result = await Runner.run(agent, "What are the contents of the file /tmp/hello.txt?")
print(result.final_output)

The weather in San Francisco is sunny!


ERROR:openai.agents:[non-fatal] Tracing: max retries reached, giving up on this batch.


The contents of the file `/tmp/hello.txt` are:

```
Hello, World!
```


ERROR:openai.agents:[non-fatal] Tracing: max retries reached, giving up on this batch.
ERROR:openai.agents:[non-fatal] Tracing: max retries reached, giving up on this batch.


## Sessions

In [39]:
from agents import Agent, Runner, SQLiteSession

# Create agent
agent = Agent(
    name="Assistant",
    instructions="Reply very concisely.",
    model=glm_model,
    model_settings=ModelSettings(include_usage=True)
)

# Create a new conversation
session = SQLiteSession(session_id="conv_123")

# Optionally resume a previous conversation by passing a conversation ID
# session = OpenAIConversationsSession(conversation_id="conv_123")

# Start conversation
result = await Runner.run(
    agent,
    "What city is the Golden Gate Bridge in?",
    session=session
)
print(result.final_output)  # "San Francisco"

# Continue the conversation
result = await Runner.run(
    agent,
    "What state is it in?",
    session=session
)
print(result.final_output)  # "California"

San Francisco.
California.


## Streaming

In [41]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=glm_model,
        model_settings=ModelSettings(include_usage=True)
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)



asyncio.run(main())

Here are 5 jokes for you:

1.  **Why don't scientists trust atoms?**
    Because they make up everything.

2.  **Why did the scarecrow win an award?**
    Because he was outstanding in his field.

3.  **I told my wife she was drawing her eyebrows too high.**
    She looked surprised.

4.  **What do you call a fake noodle?**
    An impasta.

5.  **Parallel lines have so much in common.**
    It’s a shame they’ll never meet.

ERROR:openai.agents:[non-fatal] Tracing: max retries reached, giving up on this batch.
ERROR:openai.agents:[non-fatal] Tracing: max retries reached, giving up on this batch.
